In [7]:
import pandas as pd

In [2]:
import aiohttp
import asyncio
import pandas as pd

async def fetch(session, url):
    try:
        async with session.get(url) as response:
            content_type = response.headers.get('Content-Type', '')
            content = await response.read()  # Read content asynchronously
            return url, content_type, content
    except Exception as e:
        print(f'Error fetching {url}: {e}')
        return url, None, None

async def scrape_urls(df, url_column):
    urls = df[url_column].tolist()
    async with aiohttp.ClientSession() as session:
        tasks = [fetch(session, url) for url in urls]
        results = await asyncio.gather(*tasks)

        # Prepare results to merge with the DataFrame
        results_list = []
        for url, content_type, content in results:
            if content_type:
                result_dict = {
                    'URL': url,
                    'Content-Type': content_type,
                    'Content': content
                }
                if 'application/pdf' in content_type:
                    filename = url.split("/")[-1].split("?")[0]
                    with open(filename, 'wb') as f:
                        f.write(content)
                elif 'text/html' in content_type:
                    html_content = content.decode('utf-8')
                    result_dict['HTML Content'] = html_content
                results_list.append(result_dict)
            else:
                results_list.append({
                    'URL': url,
                    'Content-Type': 'Unknown',
                    'Content': None
                })

        # Convert results to DataFrame
        results_df = pd.DataFrame(results_list)
        return results_df

In [3]:
import pandas as pd

In [4]:
ls

data_cleaning.ipynb  mle-1-assign-dataset.xlsx  test.ipynb


In [8]:
df = pd.read_excel('mle-1-assign-dataset.xlsx',sheet_name='train_data')

In [9]:
# data = df.sample(5)

In [10]:
# results_df = await asyncio.run(scrape_urls(data, 'datasheet_link'))

In [22]:
import asyncio
import logging as logger
import time

import aiohttp

# from common import headers


async def get_one(session, url):

    """getting response from single url"""

    async with session.get(url) as response:
        if response.status == 200:
            logger.warning("response individual url recieved url: %s", url)
            return await response.text()
        else:
            logger.warning(
                "response individual url recieved url: %s staus code= %d",
                url,
                response.status,
            )


async def get_all(session, urls):

    """getting response from all task"""

    try:
        tasks = []
        for url in urls:
            task = asyncio.create_task(get_one(session, url))
            tasks.append(task)
        results = await asyncio.gather(*tasks)
        logger.warning("response from all urls recieved")
        return results
    except Exception as ex:
        
        logger.warning(f"error {ex} in getting data from all urls")


async def async_transaction(urls):

    """sending asynchronous request using aiohttp"""

    start = time.time()
    try:
        async with aiohttp.ClientSession(headers=headers) as session:
            logger.warning("request send")
            data = await get_all(session, urls)
            logger.warning("aiohttp operation executed succesfully")
            end = time.time()
            logger.warning(
                "asyncronous request and response completed in : %d sec", end - start
            )
            return data
    except:
        logger.warning("error in aynchronous transaction")

In [23]:
urls = df['datasheet_link'].values.tolist()

In [25]:
urls

['https://lfillumination.com/files/specsheets/EF408B-Light-Unit.pdf',
 'https://lfillumination.com/files/specsheets/EF411B.PDF',
 'https://lfillumination.com/files/specsheets/EF407B-Light-Unit.pdf',
 'https://www.waclighting.com/storage/waclighting-images/specsheet_pdf/5221_SPSHT.pdf',
 'https://www.acuitybrands.com/api/products/getasset/healthcare-lighting/1626980/677116f7-4592-4516-9d8c-eae892e07331/hdmc.pdf?abl_version=02%2f23%2f2023+18%3a13%3a41&DOC_Type=SPEC_SHEET',
 'https://www.acuitybrands.com/api/products/getasset/holophane/1649332/4408a251-09be-4b61-813f-177b3ebcf10c/holophane-care222-hldmpc-cylinder-pendant-with-cord.pdf?abl_version=01%2f12%2f2023+16%3a54%3a24&DOC_Type=SPEC_SHEET',
 'https://lfillumination.com/files/specsheets/KATRPB-Pendant.pdf',
 'https://lfillumination.com/files/specsheets/JULRPB-Pendant.pdf',
 'https://www.focalpointlights.com/sites/default/files/FLCY3__Q.pdf',
 'https://www.signify.com/api/assets/v1/file/Signify/content/cbd5621f152243918593ab840107e6c6/

In [24]:
res = await async_transaction(urls)
print(res)

None


In [113]:
# results = asyncio.run(async_transaction(urls))
headers = {
    'authority': 'www.mcgillmicrowave.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
    'cache-control': 'no-cache',
    'pragma': 'no-cache',
    'sec-ch-ua': '"Google Chrome";v="119", "Chromium";v="119", "Not?A_Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Linux"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
}

In [114]:
def check_content_type(response,content):
    content_type = response.headers.get('Content-Type', '')
    # # Check the Content-Type
    start_bytes = content[:5]
    if 'application/pdf' in content_type or start_bytes.startswith(b'%PDF-'):
        print(f"The content is a PDF. for url{response.url}")
        return 'PDF'
    elif 'text/html' in content_type :
        print(f"The content is HTML. for url{response.url}")
        return 'HTML'
    else:
        print(f"Unknown content type : {content_type} for url: {response.url}")
        return content_type

In [115]:
def check_status(response):
    status = response.status
    if status == 200:
        return True
    elif status == 404:
        print(">>>>>>> page not found for url",response.url)
        return False
    else:
        print(f">>>>>>>>>>>> invalid status found:{status} for url:{response.url}")
        return False

In [116]:
import requests
import fitz  # PyMuPDF
from PIL import Image
import pytesseract
from io import BytesIO
import unicodedata

class PdfPipeline:
    def __init__(self,content) -> None:
        self.content = content

    def extract_text_from_pdf(self):
        pdf_stream = BytesIO(self.content)
        pdf_document = fitz.open(stream=pdf_stream, filetype="pdf")
        text = ""

        for page_num in range(len(pdf_document)):
            page = pdf_document.load_page(page_num)
            pix = page.get_pixmap()
            image = Image.open(BytesIO(pix.tobytes()))
            text += PdfPipeline.extract_text_from_image(image) + "\n"

        text = text.replace("\t"," ").replace("\n"," ")
        text = PdfPipeline.unicode_converter(text)
        text = text.replace("  "," ")
        return text

    @staticmethod
    def extract_text_from_image(image):
        return pytesseract.image_to_string(image)

    @staticmethod
    def unicode_converter(_str):
        return unicodedata.normalize('NFKD', _str).encode('ascii', errors='replace').decode().replace('?', " ")

In [117]:
from lxml import html
import unicodedata

class HtmlPipeline:
    def __init__(self,content,url) -> None:
        self.content= content
        self.url = url

    def parser(self):
        try:
            tree = html.fromstring(self.content)
        except Exception as ex:
            print(f"Exception: {ex} occurred for url:{self.url}")
            tree = ""

        text = ""
        if tree:
            for unwanted_element in tree.xpath('//script | //style'):
                unwanted_element.getparent().remove(unwanted_element)
                
            text = " ".join([str_.strip() for str_ in tree.xpath('//body//text()') if str_.strip()])
            text = text.replace("\t"," ").replace('\n'," ")
            text = HtmlPipeline.unicode_converter(text)
            text = text.replace('  '," ")
        return text

    @staticmethod
    def unicode_converter(_str):
        return unicodedata.normalize('NFKD', _str).encode('ascii', errors='replace').decode().replace('?', " ")


    def get_text(self):
        return self.parser()
            

In [148]:
import pandas as pd
import aiohttp
import asyncio
import nest_asyncio
import logging
from tqdm.asyncio import tqdm  

# Apply nest_asyncio to handle nested event loops in Jupyter notebooks
nest_asyncio.apply()

# Set up logger
logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)

# Headers for HTTP requests
# headers = {"User-Agent": "Mozilla/5.0"}

async def fetch(session, url, headers):
    """Fetch content from a single URL with custom headers."""
    try:
        async with session.get(url, headers=headers) as response:
            if check_status(response):
                logger.warning("Fetched URL: %s", url)
                content  = await response.read()
                content_type = check_content_type(response,content)

                if content_type == 'PDF':
                    pipeline = PdfPipeline(content)
                    text = pipeline.extract_text_from_pdf()
                    return url,text
                elif content_type == "HTML":
                    pipeline = HtmlPipeline(content,url)
                    text = pipeline.get_text()
                    return url,text
                else:
                    return url, content
                
            else:
                return url, ""
    except Exception as e:
        logger.warning("Error fetching URL: %s Exception: %s", url, e)
        return url, None

async def fetch_all(urls):
    """Fetch content from all URLs concurrently with custom headers."""
    tasks = []
    async with aiohttp.ClientSession() as session:
        for url in urls:
            headers['authority'] = url
            # print(headers)
            tasks.append(fetch(session, url, headers))
        results = []
        results = await asyncio.gather(*tasks)
        return results
        # for result in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc='Fetching URLs'):
        #     results.append(await result)
        # return results

def scrape_urls_from_dataframe(df, url_column):
    """Scrape URLs listed in a DataFrame column."""
    urls = df[url_column].tolist()
    # urls = ['https://mm.digikey.com/Volume0/opasdata/d220001/medias/docus/13/50CX-42.pdf'
    # ,'https://mm.digikey.com/Volume0/opasdata/d220001/medias/docus/5731/tmp_190554.tmp.pdf']
    # Run the async tasks using the existing event loop
    results = asyncio.run(fetch_all(urls))
    return results



In [150]:
# import pandas as pd
df = pd.read_csv('train_df.csv')

In [111]:
# df['datasheet_link']
# df = df.head()

In [149]:
# Sample DataFrame with URLs
# data = {'urls': ['http://example.com', 'http://example.org']}
# df = pd.DataFrame(data)

# Scrape URLs and store results
results = scrape_urls_from_dataframe(df, 'datasheet_link')

# Convert results to DataFrame
train_results_df = pd.DataFrame(results, columns=['url', 'content'])

# Display the results DataFrame
train_results_df .head()
train_results_df.to_csv('train_results_df.csv')

In [123]:
# train_results_df = pd.read_csv('train_results_df.csv')

In [125]:
# train_df = pd.read_csv('train_df.csv')

In [126]:
# train_results_df.shape

(1856, 3)

In [127]:
# train_results_df.head()

,Unnamed: 0,url,content
0,0,https://mm.digikey.com/Volume0/opasdata/d22000...,NaN
1,1,https://mm.digikey.com/Volume0/opasdata/d22000...,NaN
2,2,https://mm.digikey.com/Volume0/opasdata/d22000...,NaN
3,3,https://mm.digikey.com/Volume0/opasdata/d22000...,NaN
4,4,https://mm.digikey.com/Volume0/opasdata/d22000...,NaN


In [151]:
train_df.shape

(7022, 3)

In [153]:
train_df[train_df.duplicated()]

,datasheet_link,target_col,content
501,https://www.belfuse.com/resources/datasheets/c...,fuses,NaN
502,https://www.belfuse.com/resources/datasheets/c...,fuses,NaN
503,https://www.belfuse.com/resources/datasheets/c...,fuses,NaN
504,https://www.belfuse.com/resources/datasheets/c...,fuses,NaN
505,https://www.belfuse.com/resources/datasheets/c...,fuses,NaN
...,...,...,...
6994,https://www.microsemi.com/document-portal/doc_...,others,NaN
6998,https://www.microsemi.com/document-portal/doc_...,others,NaN
6999,https://www.microsemi.com/document-portal/doc_...,others,NaN
7019,https://www.microsemi.com/document-portal/doc_...,others,Microsemi Quality Investors Sales Contacts Par...


In [157]:
train_df.drop_duplicates(subset=['datasheet_link'],inplace=True)
train_df.to_csv('train_df.csv')


In [158]:
train_df.shape

(1197, 3)

In [131]:
train_df.drop('Unnamed: 0',inplace=True,axis=1)

In [132]:
train_results_df.drop('Unnamed: 0',inplace=True,axis=1)

In [135]:
train_df = pd.merge(train_df, train_results_df, left_on='datasheet_link', right_on='url', how='left')


In [139]:
train_df.drop('url',axis=1,inplace=True)

In [140]:
train_df.to_csv('train_df.csv',index=False)

In [141]:
test_df = pd.read_excel('mle-1-assign-dataset.xlsx',sheet_name='test_data')

In [145]:
results = scrape_urls_from_dataframe(test_df, 'datasheet_link')


The content is a PDF. for urlhttps://cdn.currentlighting.com/site/specsheet/dl_pgf1_specsht.pdf
The content is a PDF. for urlhttps://bartco.wpenginepowered.com/wp-content/uploads/2021/10/BSS745_cs_0123.pdf
The content is a PDF. for urlhttps://cdn.currentlighting.com/site/specsheet/dl_evo_spec.pdf


The content is a PDF. for urlhttps://cdn.currentlighting.com/site/specsheet/TS_Airo_spec_sheet.pdf
The content is a PDF. for urlhttps://www.signify.com/api/assets/v1/file/Signify/content/92e2537765254d99bc80ac4e01338399/U2C_2x2_grid_ceiling_mount.pdf
The content is a PDF. for urlhttps://www.assets.signify.com/is/content/Signify/e17027083e954ecd850fad4100713747
The content is a PDF. for urlhttps://www.signify.com/api/assets/v1/file/Signify/content/8edf6943506040c09834ac4e013388b6/UIW_indirect_wall.pdf


The content is a PDF. for urlhttps://lumenart.com/images/designer/wlp_specs.pdf


The content is a PDF. for urlhttps://lumenart.com/images/fabric/cyp/cyp_specs.pdf
The content is a PDF. for urlhttps://cdn.currentlighting.com/site/specsheet/lnc2%20spec%20sheet%20prismatic.pdf
>>>>>>>>>>>> invalid status found:403 for url:https://axislighting-balancedcare-production.s3.ca-central-1.amazonaws.com/wp-content/uploads/2022/04/06114330/BCTQG_44.SPEC_.pdf
>>>>>>>>>>>> invalid status found:403 for url:https://axislighting-balancedcare-production.s3.ca-central-1.amazonaws.com/wp-content/uploads/2022/04/05102136/BCTQD.SPEC_.pdf
>>>>>>>>>>>> invalid status found:403 for url:https://axislighting-balancedcare-production.s3.ca-central-1.amazonaws.com/wp-content/uploads/2022/04/06114303/BCTQG_22.SPEC_.pdf
The content is a PDF. for urlhttps://www.cooperlighting.com/api/assets/v1/file/CLS/content/ad9e025564034a3e850dad41008b7cd6/PS519075EN-312EN-300x1200-LED-Metric-SSS
The content is a PDF. for urlhttps://www.signify.com/api/assets/v1/file/Signify/content/51758d0d97da49b99323ac4e0133

>>>>>>> page not found for url https://www.louispoulsen.com/api/downloadcenter/download?type=cutsheetpdf&id=Albertslund%20Maxi%20LED%20Upgrade%20Kit-91747%20Cut%20Sheet.pdf&overridelanguage=en-US
>>>>>>> page not found for url https://www.edisonlighting.ca/products/pdf/ed_mila_plus.pdf
The content is a PDF. for urlhttps://static1.squarespace.com/static/61df533c57361a366db19973/t/64417efb635c94198dbd7a7c/1682013947658/SABIN_CUTSHEET_L60_LIT.pdf
The content is a PDF. for urlhttps://www.signify.com/api/assets/v1/file/Signify/content/17436b495a124af6bd51aa3e00d56a48/LPW7_LytePro_sconce_G3.pdf
The content is a PDF. for urlhttps://www.luminii.com/wp-content/uploads/z_special_catalog_products/products/stenos-suspended-track/specsheets/Stenos-Suspended-Track-specification.pdf
The content is a PDF. for urlhttps://www.cooperlighting.com/api/assets/v1/file/CLS/content/43e1757e7af041efbdf8ad41006fd7f8/corelite-continua-cta-susp-led-specsheet
The content is a PDF. for urlhttps://www.signify.com/api

...

>>>>>>>>>>>> invalid status found:403 for url:https://mm.digikey.com/Volume0/opasdata/d220001/medias/docus/6155/TCOM-600.pdf
>>>>>>>>>>>> invalid status found:403 for url:https://mm.digikey.com/Volume0/opasdata/d220001/medias/docus/598/M4Q.pdf
The content is a PDF. for urlhttps://www.littelfuse.com/media?resourcetype=datasheets&itemid=545bf502-c624-4172-b0a0-3a8c5f3d65be&filename=littelfuse-fuse-215sp-datasheet


The content is a PDF. for urlhttps://www.littelfuse.com/media?resourcetype=datasheets&itemid=63f61b85-5fc5-4f00-9ca9-8d59a6252d71&filename=littelfuse_fuse_154_154t_154l_154tl_datasheet.pdf
The content is a PDF. for urlhttps://www.littelfuse.com/media?resourcetype=datasheets&itemid=63f61b85-5fc5-4f00-9ca9-8d59a6252d71&filename=littelfuse_fuse_154_154t_154l_154tl_datasheet.pdf


/tmp/ipykernel_137286/599240133.py:17: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if tree:


>>>>>>>>>>>> invalid status found:403 for url:https://mm.digikey.com/Volume0/opasdata/d220001/medias/docus/2250/1000671047_Ds.pdf
>>>>>>>>>>>> invalid status found:403 for url:https://mm.digikey.com/Volume0/opasdata/d220001/medias/docus/6155/LMR-400-LLPX.pdf
>>>>>>>>>>>> invalid status found:403 for url:https://mm.digikey.com/Volume0/opasdata/d220001/medias/docus/1818/Accessories_Inlet_Rings.pdf
The content is HTML. for urlhttps://www.alphawire.com/products/cable/alpha-essentials/coaxial-cable/9213
>>>>>>>>>>>> invalid status found:403 for url:https://mm.digikey.com/Volume0/opasdata/d220001/medias/docus/657/Accessories,Finger,FilterGuards.pdf
The content is a PDF. for urlhttps://www.littelfuse.com/media?resourcetype=datasheets&itemid=533cd5cc-956c-4243-867f-6ab5a62f6ba1&filename=littelfuse_fuse_451_453_datasheet.pdf
The content is a PDF. for urlhttps://multimedia.3m.com/mws/media/1426851O/isoloss-sl-mss52.pdf
The content is a PDF. for urlhttps://www.littelfuse.com/media?resourcetype=da

>>>>>>>>>>>> invalid status found:403 for url:https://mm.digikey.com/Volume0/opasdata/d220001/medias/docus/1062/C1178.21.01.pdf
>>>>>>>>>>>> invalid status found:403 for url:https://mm.digikey.com/Volume0/opasdata/d220001/medias/docus/404/Capacitors.pdf
The content is a PDF. for urlhttps://multimedia.3m.com/mws/media/1426847O/isodamp-c-8000-mss51.pdf


In [146]:
test_results_df = pd.DataFrame(results, columns=['url', 'content'])

# Display the results DataFrame
# results_df.head()
test_results_df.to_csv('test_results_df.csv',index=False)

In [147]:
test_df = pd.merge(test_df, test_results_df, left_on='datasheet_link', right_on='url', how='left')
test_df.to_csv('test_df.csv',index=False)


In [162]:
test_df = pd.read_csv('test_df.csv')

In [164]:
test_df.drop_duplicates(subset=['datasheet_link'],inplace=True)
# test_df.to_csv('test_df.csv')

In [165]:
test_df.to_csv('test_df.csv')